In [45]:
n = 3
factor = 3
K = [Symbol(f'K_{i+1}') for i in range(n-1)]
D = Function('D')
h = Symbol('h')

In [53]:
def getFactor(expr, n):
    i = 0
    while K[i] not in expr.free_symbols:
        i += 1
    if i == n:
        return None
    e = expr.diff(K[i])
#     display(e)
    ls = e.as_ordered_factors()
#     display(ls)
    numcheck = lambda k: k.is_number
    vals = np.array(list(filter(numcheck, ls)))
    if len(vals) == 0:
        return 1
    vals = functools.reduce(lambda a, b: a * b, vals)
#     print(vals)
    return 1 / vals
    
    
def extrapolate(a, b, n):
    A = getFactor(a, n)
    B = getFactor(b, n)
    factor = B / A
    return (factor * b - a) / (factor - 1)

In [54]:
def richardsons(main_term, err_coefs, n, step, factor):
    terms =  functools.reduce(lambda a, b: a + b, [err_coefs[i] * h ** (i+1) for i in range(len(err_coefs))])
    data = np.zeros((n, n), dtype='object')
    data[0][0] = main_term(step) + terms
    for i in range(1, n):
        step /= factor
        data[0][i] = main_term(step) + terms.subs(h, step)
    
    for i in range(1, n):
        for j in range(i, n):
            data[i][j] = extrapolate(data[i-1][j-1], data[i-1][j], n)
    
    return data
data = richardsons(D,  K, n, h, factor)

In [55]:
data[-1][-1]

27*D(h/9)/16 - 3*D(h/3)/4 + D(h)/16

In [59]:
display(*data[0])

K_1*h + K_2*h**2 + D(h)

K_1*h/3 + K_2*h**2/9 + D(h/3)

K_1*h/9 + K_2*h**2/81 + D(h/9)

In [60]:
display(*data[1])

0

-K_2*h**2/3 + 3*D(h/3)/2 - D(h)/2

-K_2*h**2/27 + 3*D(h/9)/2 - D(h/3)/2

In [77]:
lstr = ''
for i in range(n):
    s = h
    for j in range(i, n):
        lstr += f' N_{i+1}({s}) &= {latex(data[i][j])}' + r'\\'
        lstr += """\n"""
        s /= factor
print(lstr)

 N_1(h) &= K_{1} h + K_{2} h^{2} + D{\left(h \right)}\\
 N_1(h/3) &= \frac{K_{1} h}{3} + \frac{K_{2} h^{2}}{9} + D{\left(\frac{h}{3} \right)}\\
 N_1(h/9) &= \frac{K_{1} h}{9} + \frac{K_{2} h^{2}}{81} + D{\left(\frac{h}{9} \right)}\\
 N_2(h) &= - \frac{K_{2} h^{2}}{3} + \frac{3 D{\left(\frac{h}{3} \right)}}{2} - \frac{D{\left(h \right)}}{2}\\
 N_2(h/3) &= - \frac{K_{2} h^{2}}{27} + \frac{3 D{\left(\frac{h}{9} \right)}}{2} - \frac{D{\left(\frac{h}{3} \right)}}{2}\\
 N_3(h) &= \frac{27 D{\left(\frac{h}{9} \right)}}{16} - \frac{3 D{\left(\frac{h}{3} \right)}}{4} + \frac{D{\left(h \right)}}{16}\\

